In [1]:
import pandas as pd 
from src.config import *

news_df = pd.read_csv(DEVELOPMENT_PATH, index_col=0, na_values=NAN_PATTERNS)

In [3]:
news_df['page_rank'].value_counts(normalize=True, dropna=False) * 100

page_rank
5    92.367214
4     4.615173
3     2.963861
2     0.053752
Name: proportion, dtype: float64

Really inbalanced

In [5]:
for rank in news_df['page_rank'].unique():
    rank_df = news_df[news_df['page_rank'] == rank]
    print(f"RANK: {rank}")
    print(rank_df['label'].value_counts(normalize=True) * 100)
    print(rank_df['source'].nunique())
    print('='*30)

RANK: 5
label
0    31.860443
5    17.665210
1    13.740510
3    13.066544
4    10.565563
2     8.903655
6     4.198076
Name: proportion, dtype: float64
1354
RANK: 3
label
2    100.0
Name: proportion, dtype: float64
2
RANK: 4
label
2    58.721560
4    20.774648
1    11.782232
3     8.721560
Name: proportion, dtype: float64
8
RANK: 2
label
2    100.0
Name: proportion, dtype: float64
2


Intuitively, rank = 2 and rank = 3 seem to convey very similar information. Merging them into a single bin could increase their statistical relevance in terms of distribution percentage, but the resulting percentage would still remain very low.

In [6]:
news_df[news_df['page_rank'] == 2]['source'].value_counts(), news_df[news_df['page_rank'] == 3]['source'].value_counts()

(source
 kuro5hin    28
 PeerFear    15
 Name: count, dtype: int64,
 source
 CNET       1280
 RedNova    1080
 Name: count, dtype: int64)

In [8]:
news_df[news_df['source'] == 'kuro5hin']['label'].value_counts(), news_df[news_df['source'] == 'PeerFear']['label'].value_counts()

(label
 2    28
 Name: count, dtype: int64,
 label
 2    15
 Name: count, dtype: int64)

In [9]:
news_df[news_df['source'] == 'CNET']['label'].value_counts(), news_df[news_df['source'] == 'RedNova']['label'].value_counts()

(label
 2    1326
 1      16
 5      15
 3      12
 Name: count, dtype: int64,
 label
 2    1959
 0     540
 3     202
 Name: count, dtype: int64)

The final decision is to create a 3 bucket column for page_rank:

- {2,3} => low
- {4} => medium
- {5} => high

and due to the fact that we're talking about an ordinal features, I will encode it as low => 0, medium => 1, high => 2